In [1]:
!pip install transformers peft datasets wandb huggingface_hub

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from datasets import load_dataset, DatasetDict
from peft import LoraConfig, get_peft_model
from huggingface_hub import notebook_login
import wandb
import torch
from datasets import load_dataset, DatasetDict
from transformers import AutoTokenizer, GPT2LMHeadModel, AutoConfig, DataCollatorForLanguageModeling, Trainer, TrainingArguments
from transformers import pipeline
import os

In [4]:
# Login to Weights and Biases and Hugging Face Hub
wandb.login()
notebook_login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [5]:
# Set seed for reproducibility
torch.manual_seed(42)

# Load dataset
dataset = load_dataset("ArtifactAI/arxiv_python_research_code")
dataset_train = dataset["train"]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Resolving data files:   0%|          | 0/26 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/1415924 [00:00<?, ? examples/s]

Loading dataset shards:   0%|          | 0/26 [00:00<?, ?it/s]

In [6]:
# Prepare datasets for training and validation
split_datasets = DatasetDict({
    "train": dataset_train.shuffle(seed=42).select(range(50000)),
    "valid": dataset_train.shuffle(seed=42).select(range(500))
})

In [7]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("huggingface-course/code-search-net-tokenizer")
context_length = 256

tokenizer_config.json:   0%|          | 0.00/265 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/789k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/448k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.34M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

In [8]:
# Tokenization function
def tokenize_data(batch):
    outputs = tokenizer(
        batch["code"],
        truncation=True,
        max_length=context_length,
        return_overflowing_tokens=True,
        return_length=True
    )
    batch_input_ids = [
        input_ids for length, input_ids in zip(outputs["length"], outputs["input_ids"]) if length == context_length
    ]
    return {"input_ids": batch_input_ids}

# Tokenize datasets
tokenized_datasets = split_datasets.map(tokenize_data, batched=True, remove_columns=split_datasets["train"].column_names)

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [9]:
# Load model
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Apply LoRA
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["c_attn", "c_proj"],  # Targeting the Conv1D layers in GPT-2
    lora_dropout=0.1,
    bias="none",
)
model = get_peft_model(model, lora_config)

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/layer.py:1091: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [10]:
from transformers import DataCollatorForLanguageModeling

# Prepare data collator with labels
tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False, return_tensors="pt"
)

In [11]:
from transformers import Trainer, TrainingArguments

# Adjust the training arguments
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/Quack/python_code_lora",
    per_device_train_batch_size=4,  # Reduced batch size
    per_device_eval_batch_size=4,
    evaluation_strategy="epoch",
    logging_steps=25,
    num_train_epochs=1,
    learning_rate=1e-3,
    save_steps=100,
    report_to="wandb",
    gradient_accumulation_steps=4,  # Gradient accumulation
    fp16=True  # Mixed precision training
)

# Initialize Trainer with the updated arguments
trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["valid"]
)

# Fine-tune the model
torch.cuda.empty_cache()  # Clear cache before training
trainer.train()
trainer.push_to_hub()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: singh-ayushk1128. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
0,3.758200,No log


Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/3.25M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/phoenix28/python_code_lora/commit/1aef72972aed8caa67c300bbb89da0a1719ebe65', commit_message='End of training', commit_description='', oid='1aef72972aed8caa67c300bbb89da0a1719ebe65', pr_url=None, pr_revision=None, pr_num=None)

In [13]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
generator = pipeline(
    "text-generation", model='phoenix28/python_code_lora', tokenizer=tokenizer, device=device
)

adapter_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

In [14]:
wandb.finish()

eval/runtime,▁
eval/samples_per_second,▁
eval/steps_per_second,▁
train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███▁
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███▁
train/grad_norm,▅▇▅▂▄▄▄▅▃▂▂▃▅▁▄█▃▄▄▄▂▅▇▂▃▃▄▇▇▂▆▇▄▃▃▂▃▁▃▇
train/learning_rate,███▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁█
train/loss,█▇▅▅▄▄▄▄▃▄▃▄▃▃▃▃▂▃▂▃▂▂▃▁▂▂▁▂▂▂▃▂▂▁▂▂▂▁▂▃
eval/runtime,27.3821
eval/samples_per_second,141.845
eval/steps_per_second,35.461


In [15]:
# Function to generate output
def generate_output(prompt):
    return generator(prompt, num_return_sequences=1)[0]["generated_text"]

In [16]:
# Example prompt
prompt = """
# create some data
x = np.random.randn(100)
y = np.random.randn(100)

# create scatter plot with x, y
"""
print(generate_output(prompt))


# create some data
x = np.random.randn(100)
y = np.random.randn(100)

# create scatter plot with x, y
�en np.random.randn(100, y


### Quantization:

GPTQ is a post-training quantization (PTQ) method for 4-bit quantization that focuses primarily on GPU inference and performance.

The idea behind the method is that it will try to compress all weights to a 4-bit quantization by minimizing the mean squared error to that weight. During inference, it will dynamically dequantize its weights to float16 for improved performance whilst keeping memory low.

In [17]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# Load LLM and Tokenizer
model_id = "phoenix28/python_code_lora"
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    trust_remote_code=False,
    revision="main"
)

# Create a pipeline
pipe = pipeline(model=model, tokenizer=tokenizer, task='text-generation')

tokenizer_config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/789k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/448k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/3.25M [00:00<?, ?B/s]

In [21]:
prompt = """
# create some data
x = np.random.randn(100)
y = np.random.randn(100)

# create scatter plot with x, y
"""
print(prompt)


# create some data
x = np.random.randn(100)
y = np.random.randn(100)

# create scatter plot with x, y



In [27]:
outputs = pipe(
    prompt,
    max_new_tokens=50,
    do_sample=True,
    temperature=0.7,
    top_k=50,
    repetition_penalty=2.0
)
print(outputs[0]["generated_text"])


# create some data
x = np.random.randn(100)
y = np.random.randn(100)

# create scatter plot with x, y
def test_error():
    if self._get('X', not '2graph':
        m1d----0[j] += 1

        # If return True: for a-n in 0 and b
                w/k
		
